# Gauss-Newton for XRD

In [1]:
from XRDXRFutils import GaussNewton,SpectraXRD,DatabaseXRD,PhaseList
from matplotlib.pyplot import xlim,ylim,xlabel,ylabel,legend,plot,fill_between
from numpy import prod,array,minimum

In [2]:
path = '/home/zdenek/Projects/pyMaXRDXRF/M491/ProfiloXRD/'
spectra = SpectraXRD().from_file(path + 'Frame0128.dat')
spectra.opt = [-1185,1950,51]

database = DatabaseXRD().read_cifs('/home/zdenek/Projects/XRDXRFutils/Database')

lazurite = database['Lazurite'][0]
hydrocerussite = database['Hydrocerussite'][0]
cinnabar = database['Cinnabar'][0]
barite = database['Barite'][0]
spinel = database['Spinel'][0]
calcite = database['Calcite'][0]

phases = PhaseList([hydrocerussite,lazurite,cinnabar,barite,spinel,calcite])

In [3]:
class PhaseSearch():
    def __init__(self,phases,spectra):
        self.g_list = [GaussNewton(phase,spectra) for phase in phases]
        
        for g in self.g_list:
            g.opt = self.g_list[0].opt
        
    def minimize_gamma(self,n=4):
        for g in self.g_list:
            for i in range(n):
                g.minimize_gamma()
                
    def overlap_area(self):
        return array([g.overlap_area() for g in self.g_list])
    
    def loss(self):
        return array([g.overlap_area() for g in self.g_list])
    
    def select(self):
        idx = self.overlap_area().argmax()
        selected = self.g_list[idx]
        
        return selected
    
    def theta_correction(self,n=4):
        for i in range(n):
            self.select().calibration()

In [4]:
phase_search = PhaseSearch(phases,spectra)
phase_search.minimize_gamma()
phase_search.select()
phase_search.theta_correction()

In [5]:
phase_search.overlap_area()

array([43.20572575, 19.71052015,  3.43219604, 18.05396628,  1.12257691,
        7.97937399])

In [34]:
%matplotlib qt

g_list[0].plot_spectra(lw=1)
g_list[0].plot('-',label='fit',lw=1)
g_list[0].phase.plot('k')

fill_between(g_list[0].theta,g_list[0].overlap(),color='r',alpha=0.33)

legend(frameon=False)
ylim(-0.05,1)
xlim(20,50)
xlabel(r'angle $\theta$')
ylabel(r'relative intensity')

NameError: name 'g_list' is not defined